In [1]:
import pandas as pd
import numpy as np

In [2]:
temp_df = pd.read_csv('NOAAGlobalTemp_v5a.csv')
temp_df.head

<bound method NDFrame.head of                time   lat    lon    z  anom
0        1880-01-01 -87.5    2.5  0.0   NaN
1        1880-01-01 -87.5    7.5  0.0   NaN
2        1880-01-01 -87.5   12.5  0.0   NaN
3        1880-01-01 -87.5   17.5  0.0   NaN
4        1880-01-01 -87.5   22.5  0.0   NaN
...             ...   ...    ...  ...   ...
4408987  2021-09-01  87.5  337.5  0.0   NaN
4408988  2021-09-01  87.5  342.5  0.0   NaN
4408989  2021-09-01  87.5  347.5  0.0   NaN
4408990  2021-09-01  87.5  352.5  0.0   NaN
4408991  2021-09-01  87.5  357.5  0.0   NaN

[4408992 rows x 5 columns]>

In [7]:
coral_df = pd.read_csv('all_coral.csv')
coral_df.head

<bound method NDFrame.head of        Unnamed: 0      age    d18O    lat     lon elevation  \
0               0  1990.00 -4.1650  11.17  -60.85        -4   
1               1  1989.00 -4.3400  11.17  -60.85        -4   
2               2  1988.00 -4.1175  11.17  -60.85        -4   
3               3  1987.00 -4.2375  11.17  -60.85        -4   
4               4  1986.00 -4.4175  11.17  -60.85        -4   
...           ...      ...     ...    ...     ...       ...   
54245         848  1219.79 -4.5600   5.87 -162.13       NaN   
54246         849  1219.88 -4.5600   5.87 -162.13       NaN   
54247         850  1219.96 -4.6100   5.87 -162.13       NaN   
54248         851  1220.04 -4.5100   5.87 -162.13       NaN   
54249         852  1220.13 -4.4700   5.87 -162.13       NaN   

                    species  
0      Montastaea faveolata  
1      Montastaea faveolata  
2      Montastaea faveolata  
3      Montastaea faveolata  
4      Montastaea faveolata  
...                     ...  
542

In [10]:
temp_df.lat.unique()

array([-87.5, -82.5, -77.5, -72.5, -67.5, -62.5, -57.5, -52.5, -47.5,
       -42.5, -37.5, -32.5, -27.5, -22.5, -17.5, -12.5,  -7.5,  -2.5,
         2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,  42.5,
        47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,  87.5])

In [11]:
temp_df.lon.unique()

array([  2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,  42.5,
        47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,  87.5,
        92.5,  97.5, 102.5, 107.5, 112.5, 117.5, 122.5, 127.5, 132.5,
       137.5, 142.5, 147.5, 152.5, 157.5, 162.5, 167.5, 172.5, 177.5,
       182.5, 187.5, 192.5, 197.5, 202.5, 207.5, 212.5, 217.5, 222.5,
       227.5, 232.5, 237.5, 242.5, 247.5, 252.5, 257.5, 262.5, 267.5,
       272.5, 277.5, 282.5, 287.5, 292.5, 297.5, 302.5, 307.5, 312.5,
       317.5, 322.5, 327.5, 332.5, 337.5, 342.5, 347.5, 352.5, 357.5])

Since this temperature dataset only has gridded increments of 5 degrees across both latitude and longitude, we need to find the closest gridded temp lat & long for each coral lat & long.

We will be building 5x5 lat/lon grids around each data point!

In [30]:
def get_5x5_grid(og_lat, og_lon, base=2.5):
    lat1 = base * round((og_lat-2.5)/base)
    lat2 = base * round((og_lat+2.5)/base)
    
    print(str(lat1), str(lat2))

    if lat2-2.5 < og_lat:
        lat2 += 2.5
        lat1 += 2.5
    else:
        lat2 -= 2.5
        lat1 -= 2.5

    print(str(lat1), str(lat2)) 

    if (lat1 % 5 )== 0:
        if lat1 < 0:
            lat1 += 2.5
            lat2 += 2.5
        else: 
            lat1 -= 2.5
            lat2 -= 2.5

    print(str(lat1), str(lat2)) 

    rounded_lat = base * round(og_lat/base)
    rounded_lon = base * round(og_lon/base)
    
    
    return lat1, lat2#, lon1, lon2

In [32]:
get_5x5_grid(86, 0)

82.5 87.5
85.0 90.0
87.5 92.5


(87.5, 92.5)

In [16]:
myround(6)

#base_cases
if rounded_lat == -87.5:
    lat1, lat2 = -87.5, -82.5
elif rounded_lat == 87.5:
    lat1, lat2 = 82.5, 87.5
# regular
else:
    if (rounded_lat // 5 )==0:
        lat1 = rounded_lat - 2.5
        lat2 = rounded_lat + 2.5
    else: 

5.0